In [ ]:
%pip install -qq docling docling-core
%pip install pymupdf
%pip install langchain-openai
%pip install langchain
%pip install qdrant-client
%pip install -U langchain-community
%pip install langchain_huggingface

In [ ]:
# reference documents
# https://www.analyticsvidhya.com/blog/2025/03/enhancing-multimodal-rag-capabilities-using-docling/
# https://medium.com/the-ai-forum/building-an-advanced-rag-pipeline-using-docling-groq-ollama-with-glider-evaluation-b91c290b0c84

In [ ]:
import os
import fitz
from langchain_openai import OpenAIEmbeddings
from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI
from docling.document_converter import DocumentConverter, PdfFormatOption
from docling.datamodel.base_models import InputFormat
from docling.datamodel.pipeline_options import PdfPipelineOptions

In [ ]:
from google.colab import userdata
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [ ]:
!wget 'https://s21.q4cdn.com/399680738/files/doc_news/Meta-Reports-Second-Quarter-2024-Results-2024.pdf' -O './doc1.pdf'

--2025-06-28 16:59:45--  https://s21.q4cdn.com/399680738/files/doc_news/Meta-Reports-Second-Quarter-2024-Results-2024.pdf
Resolving s21.q4cdn.com (s21.q4cdn.com)... 68.70.205.4, 68.70.205.1, 68.70.205.3, ...
Connecting to s21.q4cdn.com (s21.q4cdn.com)|68.70.205.4|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 195613 (191K) [application/pdf]
Saving to: ‘./doc1.pdf’

./doc1.pdf          100%[===================>] 191.03K  --.-KB/s    in 0.1s    

2025-06-28 16:59:46 (1.35 MB/s) - ‘./doc1.pdf’ saved [195613/195613]



# Document reading

In [ ]:
pdf_pipeline_options = PdfPipelineOptions(do_ocr=False,generate_picture_images=True,)
format_options = {InputFormat.PDF: PdfFormatOption(pipeline_options=pdf_pipeline_options)}
converter = DocumentConverter(format_options=format_options)

sources = ["/content/doc1.pdf",]
conversions = {source: converter.convert(source=source).document for source in sources}

In [ ]:
# conversions[sources[0]]

In [ ]:
from docling_core.transforms.chunker.hybrid_chunker import HybridChunker
from docling_core.types.doc.document import TableItem
from langchain_core.documents import Document


doc_id = 0
texts= []

for source, docling_document in conversions.items():
    for chunk in HybridChunker(tokenizer=embeddings_tokenizer).chunk(docling_document):
        items = chunk.meta.doc_items
        if len(items) == 1 and isinstance(items[0], TableItem):
            continue # we will process tables later
        refs = " ".join(map(lambda item: item.get_ref().cref, items))
        text = chunk.text
        document = Document(page_content=text,metadata={"doc_id": (doc_id:=doc_id+1),"source": source,"ref": refs,},)
        texts.append(document)

print(f"{len(texts)} text document chunks created")

Token indices sequence length is longer than the specified maximum sequence length for this model (875 > 512). Running this sequence through the model will result in indexing errors


22 text document chunks created


In [ ]:
from docling_core.types.doc.labels import DocItemLabel

doc_id = len(texts)
tables= []

for source, docling_document in conversions.items():
    for table in docling_document.tables:
        if table.label in [DocItemLabel.TABLE]:
            ref = table.get_ref().cref
            text = table.export_to_markdown()
            document = Document(
                page_content=text,
                metadata={
                    "doc_id": (doc_id:=doc_id+1),
                    "source": source,
                    "ref": ref
                },
            )
            tables.append(document)


print(f"{len(tables)} table documents created")

11 table documents created


In [ ]:
for i in range(len(tables)):
  print("\n###########################NEW TABLE##########################\n")
  print(tables[i].page_content)


###########################NEW TABLE##########################

|                                                       | Three Months Ended June 30,   | Three Months Ended June 30,   |         |
|-------------------------------------------------------|-------------------------------|-------------------------------|---------|
| In millions, except percentages and per share amounts | 2024                          | 2023                          | %Change |
| Revenue                                               | $ 39,071                      | $ 31,999                      | 22%     |
| Costs and expenses                                    | 24,224                        | 22,607                        | 7%      |
| Income from operations                                | $ 14,847                      | $ 9,392                       | 58%     |
| Operating margin                                      | 38%                           | 29%                           |         |
| Provision

In [ ]:
docling_document.pictures

[]

In [ ]:
combined_docs = texts + tables

In [ ]:
for i in range(len(combined_docs)):
  print(combined_docs[i].metadata["ref"])

#/texts/0
#/texts/2 #/texts/3
#/tables/0
#/texts/6 #/texts/7 #/texts/8 #/texts/9 #/texts/10 #/texts/11 #/texts/12 #/texts/13 #/texts/14 #/texts/15 #/texts/16
#/texts/18 #/texts/19 #/texts/20 #/texts/21 #/texts/22 #/texts/23
#/texts/25 #/texts/27 #/texts/28
#/texts/30
#/texts/32
#/texts/34 #/texts/35 #/texts/36 #/texts/37 #/texts/38 #/texts/40
#/texts/41
#/texts/43 #/texts/44 #/texts/46 #/texts/47 #/texts/48 #/texts/49 #/texts/50
#/texts/53 #/texts/54
#/tables/2 #/texts/56
#/texts/59 #/texts/60
#/tables/3 #/tables/4
#/texts/63 #/texts/64
#/tables/6
#/texts/68 #/texts/69
#/texts/71 #/texts/72
#/texts/74 #/texts/75 #/tables/8 #/texts/76 #/tables/9
#/texts/79 #/texts/80
#/tables/10 #/texts/81
#/tables/0
#/tables/1
#/tables/2
#/tables/3
#/tables/4
#/tables/5
#/tables/6
#/tables/7
#/tables/8
#/tables/9
#/tables/10


# Embedding

In [ ]:
from langchain_openai import OpenAIEmbeddings

embedding_encoder = OpenAIEmbeddings(
    model="text-embedding-3-small",
    openai_api_key=OPENAI_API_KEY
)

# from langchain_huggingface.embeddings import HuggingFaceEmbeddings
# from transformers import *

# embeddings_model_path = "ibm-granite/granite-embedding-30m-english"
# embeddings_hf_model = HuggingFaceEmbeddings(model_name=embeddings_model_path,)

# Qdrant

In [ ]:
embeddings = embedding_encoder.embed_documents(
    texts=[t.page_content for t in combined_docs]
)

In [ ]:
from qdrant_client.models import PointStruct
import uuid

points = []

for idx, document in enumerate(combined_docs):
    vector = embeddings[idx]
    if vector is None:
        continue  # Skip unembedded

    point = PointStruct(
        id=str(uuid.uuid4()),
        vector=vector,
        payload={
            "text": document.page_content,
            "metadata": document.metadata,
        }
    )
    points.append(point)

In [ ]:
from qdrant_client import QdrantClient

qdrant_cloud_key = userdata.get('QDRANT_CLOUD_KEY')

client = QdrantClient(
    url="https://01f14d7a-9e05-4804-bfd3-a69b4c3a97da.us-west-1-0.aws.cloud.qdrant.io:6333",
    api_key=qdrant_cloud_key,
)

print(client.get_collections())

collections=[CollectionDescription(name='meta_report'), CollectionDescription(name='meta_report_tables')]


In [ ]:
from qdrant_client.models import VectorParams, Distance

collection_name = "meta_report_tables_docling_v1"

if collection_name not in [col.name for col in client.get_collections().collections]:
    client.recreate_collection(
        collection_name=collection_name,
        vectors_config=VectorParams(size=len(points[0].vector), distance=Distance.COSINE)
    )

/tmp/ipython-input-49-3941613558.py:6: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  client.recreate_collection(


In [ ]:
client.upsert(
    collection_name=collection_name,
    points=points
)

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

# RAG Pipeline

## Retreiver

In [ ]:
from langchain.vectorstores import Qdrant
from langchain.embeddings.openai import OpenAIEmbeddings  # or any embedding model you're using

collection_name = "meta_report_tables_docling_v1"

embedding_function = OpenAIEmbeddings(model="text-embedding-3-small")

vecdb = Qdrant(
    client=client,
    collection_name=collection_name,
    embeddings=embedding_function,
    content_payload_key="text"
)

/tmp/ipython-input-53-218902449.py:6: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embedding_function = OpenAIEmbeddings(model="text-embedding-3-small")
/tmp/ipython-input-53-218902449.py:8: LangChainDeprecationWarning: The class `Qdrant` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-qdrant package and should be used instead. To use it run `pip install -U :class:`~langchain-qdrant` and import as `from :class:`~langchain_qdrant import Qdrant``.
  vecdb = Qdrant(


In [ ]:
# # At the end of 2023, what was the value of Meta's Goodwill assets?
# results = vecdb.similarity_search("what is the 2024 GAAP advertising Revenue in the three months \
#      ended June 30th? What about net cash by operating activies", k=5)
# for doc in results:
#     print(doc.page_content)

## Re-ranker

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

tokenizer = AutoTokenizer.from_pretrained("BAAI/bge-reranker-v2-m3")
reranker_model = AutoModelForSequenceClassification.from_pretrained("BAAI/bge-reranker-v2-m3").to(device)

def rerank(query, docs, top_n=5):
    pairs = [(query, doc.page_content) for doc in docs]
    inputs = tokenizer.batch_encode_plus(pairs, padding=True, truncation=True, return_tensors="pt").to(device)

    with torch.no_grad():
        scores = reranker_model(**inputs).logits.squeeze(-1)

    ranked = sorted(zip(docs, scores), key=lambda x: x[1], reverse=True)
    return [doc for doc, _ in ranked[:top_n]]

tokenizer_config.json: 0.00B [00:00, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/795 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.27G [00:00<?, ?B/s]

## Query Analysis

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
import os
from pydantic import BaseModel
from typing import List
from langchain.output_parsers import PydanticOutputParser

class QueryList(BaseModel):
  queries: List[str]

class OptimizedQuery(BaseModel):
  query: str

class Document(BaseModel):
  hypothetical_document: str

class QueryAnalysis:
    def __init__(self, model_name="gpt-4o-mini", temperature=0):
        self.llm = ChatOpenAI(model_name=model_name, temperature=temperature)
        self.query_list_parser = PydanticOutputParser(pydantic_object=QueryList)
        self.single_query_parser = PydanticOutputParser(pydantic_object=OptimizedQuery)
        self.document_parser = PydanticOutputParser(pydantic_object=Document)

    def query_expansion(self, query: str, num_expansions: int = 3) -> list:
        prompt = PromptTemplate(
            input_variables=["query", "num_expansions","format_instructions"],
            template="""
            Expand the following query into {num_expansions} diverse but related queries.

            Original query: "{query}"

            {format_instructions}
            """)
        chain = prompt | self.llm | self.query_list_parser
        result = chain.invoke({"query": query,
                               "num_expansions": num_expansions,
                               "format_instructions": self.query_list_parser.get_format_instructions()})
        return self._parse_bullets(result.content)

    def query_decomposition(self, query: str) -> list:
        prompt = PromptTemplate(
            input_variables=["query","format_instructions"],
            template="""
            Decompose the following complex query into a sequence of simpler sub-questions:

            Complex query: "{query}"

            {format_instructions}
            """)
        chain = prompt | self.llm
        result = chain.invoke({"query": query,
                               "format_instructions": self.query_list_parser.get_format_instructions()})
        return self._parse_bullets(result.content)

    def hyde(self, query: str) -> str:
        prompt = PromptTemplate(
            input_variables=["query","format_instructions"],
            template="""
            Generate a hypothetical document that might answer the following query.

            Query: "{query}"

            {format_instructions}
            """)
        chain = prompt | self.llm
        result = chain.invoke({"query": query,
                               "format_instructions": self.document_parser.get_format_instructions()})
        return result.content.strip()

    def query_optimization(self, query: str, history: str) -> str:
        prompt = PromptTemplate(
            input_variables=["history", "query","format_instructions"],
            template="""
                Given the following chat history and a follow-up question, rewrite the question so that it is self-contained.

                Chat history:
                {history}

                Follow-up question:
                {query}

                {format_instructions}"""
            )

        chain = prompt | self.llm
        optimized_query = chain.invoke({
            "history": history,
            "query": query,
            "format_instructions": self.single_query_parser.get_format_instructions()
        }).content.strip()

        print("Optimized Query:", optimized_query)


## RAG

In [ ]:
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI

prompt_template = PromptTemplate(
    input_variables=["context", "question"],
    template="""
        You are an intelligent assistant. Use the following context to answer the question at the end.

        Context:
        {context}

        Question:
        {question}

        Answer:"""
        )

llm = ChatOpenAI(model_name="gpt-4o-mini")
qa_chain = prompt_template | llm

import time
def rag(query: str, k: int = 10, top_n: int=5):
    retrieved_docs = vecdb.similarity_search(query, k=k)
    # retrieved_docs = rerank(query, retrieved_docs, top_n=top_n)
    context = "\n\n".join([doc.page_content for doc in retrieved_docs])

    result = qa_chain.invoke(input={"context":context, "question":query})

    return {
        "answer": result.content,
        "token_usage":result.response_metadata["token_usage"],
        "retrieved_docs": retrieved_docs
    }

In [ ]:
# Example usage
query = "what is the 2024 GAAP advertising Revenue in the three months ended June 30th? What about net cash by operating activies"
response = rag(query,1,1)
print("Answer:", response["answer"])

## RAG with memory

In [ ]:
from langchain.memory import ConversationBufferMemory, ConversationBufferWindowMemory
from langchain.prompts import PromptTemplate

prompt_template = PromptTemplate(
    input_variables=["context", "question", "chat_history"],
    template="""
          You are an intelligent assistant. Use the chat history and the following context to answer the user's question.

          Chat history:
          {chat_history}

          Context:
          {context}

          Question:
          {question}

          Answer:
          """
        )

memory = ConversationBufferWindowMemory( k=5,  memory_key="chat_history" )
qa_chain_with_memory = LLMChain(
    llm=ChatOpenAI(model_name="gpt-4o-mini"),
    prompt=prompt_template,
    memory=memory
)

## RAG with cache
- Query only cache
- Query + Doc cache


| Feature             | Query-Only Cache            | Query + Doc Hash Cache        |
| ------------------- | --------------------------- | ----------------------------- |
| Simplicity          | ✅ Easy                      | ❌ Moderate (needs hashing)    |
| Accuracy            | ❌ May become stale          | ✅ Valid per retrieved docs    |
| Cache Hit Rate      | ✅ Higher                    | ❌ Lower (if retriever varies) |
| Token/Cost Saving   | ✅ Strong                    | ✅ Moderate                    |
| Handles doc changes | ❌ No                        | ✅ Yes                         |
| Use case            | Fast prototypes, stable RAG | Research, evaluation, prod    |


# Evaluation

In [ ]:
%pip install ragas
%pip install qdrant-client
%pip install langchain-openai
%pip install langchain
%pip install -U langchain-community
%pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.9/190.9 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 61.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.4/70.4 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 329.0/329.0 kB 17.8 MB/s eta 0:00:00


In [ ]:
%pip install rapidfuzz

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 54.1 MB/s eta 0:00:00


In [ ]:
import os
from google.colab import userdata

os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")
OPENAI_API_KEY = userdata.get("OPENAI_API_KEY")

## Fetch text chunks from Qdrant

In [ ]:
import uuid
from qdrant_client import QdrantClient
from qdrant_client.models import Filter, FieldCondition, MatchValue, SearchParams
from langchain.chat_models import ChatOpenAI
from langchain.output_parsers import PydanticOutputParser
from langchain.prompts import PromptTemplate
from pydantic import BaseModel, Field
from google.colab import userdata

qdrant_cloud_key = userdata.get('QDRANT_CLOUD_KEY')
client = QdrantClient(
    url="https://01f14d7a-9e05-4804-bfd3-a69b4c3a97da.us-west-1-0.aws.cloud.qdrant.io:6333",
    api_key=qdrant_cloud_key
)

collection_name = "meta_report_tables_docling_v1"

points = client.scroll(
    collection_name=collection_name,
    scroll_filter=None,
    limit=100,
)

In [ ]:
points = points[0]

In [ ]:
from langchain_core.documents import Document
docs = []

for point in points:
    docs.append(Document(
                page_content=point.payload["text"],
                metadata=point.payload["metadata"]
            ))

## Create synthetic data

In [ ]:
from langchain_core.runnables import RunnableLambda
from langchain.output_parsers import PydanticOutputParser
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI
from typing import List
from pydantic import BaseModel, Field

class QAPair(BaseModel):
    question: str = Field(..., description="A question based on the context")
    answer: str = Field(..., description="The answer to the question based on the context")

class QAPairList(BaseModel):
    context: str = Field(..., description="The document context")
    context_id: str = Field(..., description="Unique identifier of the context")
    qa_pairs: List[QAPair] = Field(..., description="A list of QA pairs generated from the context")

parser = PydanticOutputParser(pydantic_object=QAPairList)

prompt = PromptTemplate.from_template(
    """You are a QA generator.

    Context:
    {context}

    Generate 3 diverse and relevant question-answer pairs from the above context.
    Format:
    {format_instructions}
    """
)

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.2)

data_generator_chain = (
    prompt |
    llm |
    parser
)

synthetic_data = []

for point in points:
    context_text = point.payload["text"]
    context_id = str(point.id)

    result = data_generator_chain.invoke({
        "context": context_text,
        "format_instructions": parser.get_format_instructions()
    })

    result.context = context_text
    result.context_id = context_id
    synthetic_data.append(result)
    break

In [ ]:
print(synthetic_data[0].qa_pairs)

[QAPair(question='What was the total current liabilities on June 30, 2024?', answer='The total current liabilities on June 30, 2024 was 27,004.'), QAPair(question='How much did long-term income taxes increase from December 31, 2023 to June 30, 2024?', answer='Long-term income taxes increased from 7,514 on December 31, 2023 to 7,897 on June 30, 2024, an increase of 383.'), QAPair(question="What is the total stockholders' equity as of June 30, 2024?", answer="The total stockholders' equity as of June 30, 2024 is not provided in the context.")]


In [ ]:
all_qa = []
for item in synthetic_data:
    for qa in item.qa_pairs:
        all_qa.append({
            "question": qa.question,
            "answer": qa.answer,
            "context": item.context,
        })

In [ ]:
from datasets import Dataset

hf_dataset = Dataset.from_list(all_qa)

In [ ]:
hf_dataset.push_to_hub("JC-24/meta-record-rag-synthetic-dataset", token=userdata.get("HF_TOKEN"))

Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
hf_dataset

Dataset({
    features: ['question', 'answer', 'context'],
    num_rows: 3
})

In [ ]:
hf_dataset[0]

{'question': 'What was the total current liabilities on June 30, 2024?',
 'answer': 'The total current liabilities on June 30, 2024 was 27,004.',
 'context': "and other current liabilities, December 31, 2023 = 25,488. Total current liabilities, June 30, 2024 = 27,004. Total current liabilities, December 31, 2023 = 31,960. Operating lease liabilities, non-current, June 30, 2024 = 17,685. Operating lease liabilities, non-current, December 31, 2023 = 17,226. Long-term debt, June 30, 2024 = 18,389. Long-term debt, December 31, 2023 = 18,385. Long-term income taxes, June 30, 2024 = 7,897. Long-term income taxes, December 31, 2023 = 7,514. Other liabilities, June 30, 2024 = 2,500. Other liabilities, December 31, 2023 = 1,370. Total liabilities, June 30, 2024 = 73,475. Total liabilities, December 31, 2023 = 76,455. Commitments and contingencies, June 30, 2024 = . Commitments and contingencies, December 31, 2023 = . Stockholders' equity:, June 30, 2024 = . Stockholders' equity:, December 31, 2

## Create synthetic data using RAGAS

In [ ]:
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings

generator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4o-mini"))
generator_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings(
    model="text-embedding-3-small",
    openai_api_key=OPENAI_API_KEY
))

In [ ]:
import ragas
ragas.__version__

'0.2.15'

In [ ]:
from ragas.testset.synthesizers import default_query_distribution
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm, embedding_model=generator_embeddings)
query_distribution = default_query_distribution(generator_llm)
testset_2 = generator.generate_with_langchain_docs(docs,testset_size=10, query_distribution=query_distribution)

Applying SummaryExtractor:   0%|          | 0/21 [00:00<?, ?it/s]

Applying CustomNodeFilter:   0%|          | 0/33 [00:00<?, ?it/s]

Applying [EmbeddingExtractor, ThemesExtractor, NERExtractor]:   0%|          | 0/87 [00:00<?, ?it/s]

Applying [CosineSimilarityBuilder, OverlapScoreBuilder]:   0%|          | 0/2 [00:00<?, ?it/s]

Generating Scenarios:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/12 [00:00<?, ?it/s]

,user_input,reference_contexts,reference,synthesizer_name
0,What are the total current liabilities as of J...,"[and other current liabilities, December 31, 2...","Total current liabilities as of June 30, 2024,...",single_hop_specifc_query_synthesizer
1,What is the significance of the U.S. dollor in...,[To supplement our condensed consolidated nanc...,The U.S. dollar is significant in financial re...,single_hop_specifc_query_synthesizer
2,Wut is the revenue for Reality Labs in the thr...,[(In millions)\n(Unaudited)\nThree Months Ende...,The revenue for Reality Labs in the three mont...,single_hop_specifc_query_synthesizer
3,"What cash paid for income taxes on June 30, 2024?",[| ...,"$ 5,929",single_hop_specifc_query_synthesizer
4,What is the expected advertising revenue and t...,[<1-hop>\n\n| Advertising |...,The expected advertising revenue for the third...,multi_hop_abstract_query_synthesizer
5,Wht was the revenue for the three months ended...,"[<1-hop>\n\n30,.2023 = (12,957). Principal pay...",The revenue for the three months ended June 30...,multi_hop_abstract_query_synthesizer
6,What was the revenue growth percentage for the...,"[<1-hop>\n\nRevenue, Three Months Ended June 3...",The revenue growth percentage for the three mo...,multi_hop_abstract_query_synthesizer
7,What is the expected revenue growth for the th...,[<1-hop>\n\nWe expect third quarter 2024 total...,The expected revenue for the third quarter 202...,multi_hop_abstract_query_synthesizer
8,What was the total income from operations for ...,[<1-hop>\n\n(In millions)\n(Unaudited)\nThree ...,"In the three months ended June 30, the total i...",multi_hop_specific_query_synthesizer
9,What are the expected financial impacts on Rea...,[<1-hop>\n\nWe expect third quarter 2024 total...,"For the third quarter of 2024, total revenue i...",multi_hop_specific_query_synthesizer


In [ ]:
testset_2 = testset_2.to_pandas()
testset_2.head()

,user_input,reference_contexts,reference,synthesizer_name
0,What are the total current liabilities as of J...,"[and other current liabilities, December 31, 2...","Total current liabilities as of June 30, 2024,...",single_hop_specifc_query_synthesizer
1,What is the significance of the U.S. dollor in...,[To supplement our condensed consolidated nanc...,The U.S. dollar is significant in financial re...,single_hop_specifc_query_synthesizer
2,Wut is the revenue for Reality Labs in the thr...,[(In millions)\n(Unaudited)\nThree Months Ende...,The revenue for Reality Labs in the three mont...,single_hop_specifc_query_synthesizer
3,"What cash paid for income taxes on June 30, 2024?",[| ...,"$ 5,929",single_hop_specifc_query_synthesizer
4,What is the expected advertising revenue and t...,[<1-hop>\n\n| Advertising |...,The expected advertising revenue for the third...,multi_hop_abstract_query_synthesizer


In [ ]:
testset_2

,user_input,reference_contexts,reference,synthesizer_name
0,What are the total current liabilities as of J...,"[and other current liabilities, December 31, 2...","Total current liabilities as of June 30, 2024,...",single_hop_specifc_query_synthesizer
1,What is the significance of the U.S. dollor in...,[To supplement our condensed consolidated nanc...,The U.S. dollar is significant in financial re...,single_hop_specifc_query_synthesizer
2,Wut is the revenue for Reality Labs in the thr...,[(In millions)\n(Unaudited)\nThree Months Ende...,The revenue for Reality Labs in the three mont...,single_hop_specifc_query_synthesizer
3,"What cash paid for income taxes on June 30, 2024?",[| ...,"$ 5,929",single_hop_specifc_query_synthesizer
4,What is the expected advertising revenue and t...,[<1-hop>\n\n| Advertising |...,The expected advertising revenue for the third...,multi_hop_abstract_query_synthesizer
5,Wht was the revenue for the three months ended...,"[<1-hop>\n\n30,.2023 = (12,957). Principal pay...",The revenue for the three months ended June 30...,multi_hop_abstract_query_synthesizer
6,What was the revenue growth percentage for the...,"[<1-hop>\n\nRevenue, Three Months Ended June 3...",The revenue growth percentage for the three mo...,multi_hop_abstract_query_synthesizer
7,What is the expected revenue growth for the th...,[<1-hop>\n\nWe expect third quarter 2024 total...,The expected revenue for the third quarter 202...,multi_hop_abstract_query_synthesizer
8,What was the total income from operations for ...,[<1-hop>\n\n(In millions)\n(Unaudited)\nThree ...,"In the three months ended June 30, the total i...",multi_hop_specific_query_synthesizer
9,What are the expected financial impacts on Rea...,[<1-hop>\n\nWe expect third quarter 2024 total...,"For the third quarter of 2024, total revenue i...",multi_hop_specific_query_synthesizer


In [ ]:
from datasets import Dataset

hf_dataset = Dataset.from_pandas(testset_2)

In [ ]:
hf_dataset.push_to_hub("JC-24/meta-record-ragas-synthetic-dataset", token=userdata.get("HF_TOKEN"))

Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
for in_ in testset_2[testset_2["synthesizer_name"]=="multi_hop_abstract_query_synthesizer"]["user_input"]:
  print(in_)

What is the expected advertising revenue and total revenue guidance for the third quarter of 2024?
Wht was the revenue for the three months ended June 30, 2024, and how does it compare to the revenue for the same period in 2023?
What was the revenue growth percentage for the three months ended June 30, 2024, and how does it relate to the total assets reported as of June 30, 2024?
What is the expected revenue growth for the third quarter 2024 compared to the previous year based on the revenue guidance provided?


## Ragas

In [ ]:
dataset_path="JC-24/meta-record-ragas-synthetic-dataset"
hf_dataset = load_dataset(dataset_path, token=userdata.get("HF_TOKEN"))

In [ ]:
from datasets import Dataset, load_dataset

ragas_dataset = []
for item in hf_dataset:
  query = item["user_input"]
  response = rag(query, k=3)
  retrieved_docs = response["retrieved_docs"]
  answer = response["answer"]
  ragas_dataset.append(
      {
          "question": query,
          "ground_truth": item["reference"],
          "context": item["reference_contexts"],
          "answer":answer,
          "retrieved_contexts":[d.page_content for d in retrieved_docs]
      }
  )
ragas_dataset_hf = Dataset.from_list(ragas_dataset)

In [ ]:
import pandas as pd

ragas_df = pd.DataFrame(ragas_dataset)
ragas_df.to_csv("rag_evaluation_dataset_03.csv", index=False)

In [ ]:
len(ragas_df["retrieved_contexts"].iloc[0])

3

# MLFlow

In [ ]:
%pip install mlflow
%pip install dagshub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.0/261.0 kB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.9/139.9 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.7/13.7 MB 89.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.2/85.2 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.3/74.3 kB 6.6 MB/s eta 0:00:00


In [ ]:
import openai
import os
import pandas as pd
import dagshub
import mlflow

dagshub.init(
    repo_owner='janvichokshi1998',
    repo_name='rags-evaluation-repo',
    mlflow=True
)

mlflow.set_tracking_uri("https://dagshub.com/janvichokshi1998/rags-evaluation-repo.mlflow")

In [ ]:
import mlflow
import pandas as pd
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_precision,
    context_recall
)
from ragas import evaluate
from datasets import Dataset

def run_evaluation_pipeline(dataset_path, run_name):
  df = pd.read_csv(dataset_path)  # Or pd.read_json()
  df["context"] = df["context"].apply(lambda x: eval(x))
  df["retrieved_contexts"] = df["retrieved_contexts"].apply(lambda x: eval(x))

  ragas_dataset = Dataset.from_pandas(df)

  metric_list = [answer_relevancy, context_precision, context_recall, faithfulness]
  mlflow.set_experiment("RAGAS Evaluation Meta Report")
  with mlflow.start_run(run_name=run_name):
      mlflow.log_param("dataset", dataset_path)
      mlflow.log_param("num_samples", len(df))
      # llm = ChatOpenAI(model_name="gpt-4o-mini")
      results = evaluate(ragas_dataset, metrics=metric_list)
      for metric in metric_list:
          score = sum(results[metric.name])/len(results[metric.name])
          mlflow.log_metric(metric.name, score)

      results_df = pd.DataFrame(results.scores)
      results_df.to_csv("results.csv", index=False)
      mlflow.log_artifact("results.csv","results.csv")
      artifact_path = dataset_path.split("/")[-1]
      mlflow.log_artifact(dataset_path, artifact_path)

      print("Evaluation completed and logged to MLflow.")

In [ ]:
run_evaluation_pipeline("rag_evaluation_dataset_03.csv","gpt-4o-mini_text-embedding-3-small_k3")

Evaluating:   0%|          | 0/48 [00:00<?, ?it/s]

Evaluation completed and logged to MLflow.
🏃 View run gpt-4o-mini_text-embedding-3-small_k3 at: https://dagshub.com/janvichokshi1998/rags-evaluation-repo.mlflow/#/experiments/0/runs/2e1dd480586e4039ad97090efea47f0a
🧪 View experiment at: https://dagshub.com/janvichokshi1998/rags-evaluation-repo.mlflow/#/experiments/0


# Scaling